In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from datetime import datetime
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

import os
import sys
# sys.path.insert(1, os.path.join(sys.path[0], '..'))
os.chdir('..')

import dataset
from model_part import conv2d
from model_part import fc

In [2]:
flags = tf.app.flags
FLAGS = flags.FLAGS
today = datetime.strftime(datetime.now(), '%d%m%y')
flags.DEFINE_integer('batch_size', 8, 'Batch size')
flags.DEFINE_string('train_file', 'train.csv', 'Train file')

In [3]:
def model(inputs, is_training, reuse=False, keep_drop=0.5, trainable=True,
          debug=False):
    """Create an upscaled Eigen coarse model.

    inputs.get_shape() = TensorShape([Dimension(8), Dimension(228),
                                      Dimension(304), Dimension(3)])
    """
    # Normalize.
    inputs = (inputs - dataset.IMAGES_MEAN) * dataset.IMAGES_ISTD
    net = conv2d('conv1', inputs, [11, 11, 3, 96], [96], [1, 4, 4, 1],
                 padding='VALID', reuse=reuse, trainable=trainable, wd=0.0005,
                 activation=None)
    if debug:
        print('%s \t\t%s' % (net.name, net.get_shape()))
    net = tf.layers.batch_normalization(net, axis=3, training=is_training,
                                        name='conv1_bn')
    if debug:
        print('%s%s' % (net.name, net.get_shape()))
    net = tf.nn.max_pool(net, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1],
                         padding='VALID', name='pool1')
    if debug:
        print('%s \t\t%s' % (net.name, net.get_shape()))
    net1 = tf.nn.relu(net, name='relu1')
    if debug:
        print('%s \t\t%s' % (net1.name, net1.get_shape()))

    net = conv2d('conv2', net1, [5, 5, 96, 256], [256], [1, 1, 1, 1],
                 padding='SAME', reuse=reuse, trainable=trainable, wd=0.0005,
                 activation=None)
    if debug:
        print('%s \t\t%s' % (net.name, net.get_shape()))
    net = tf.layers.batch_normalization(net, axis=3, training=is_training,
                                        name='conv2_bn')
    if debug:
        print('%s%s' % (net.name, net.get_shape()))
    net = tf.nn.max_pool(net, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1],
                         padding='VALID', name='pool2')
    if debug:
        print('%s \t\t%s' % (net.name, net.get_shape()))
    net2 = tf.nn.relu(net, name='relu2')
    if debug:
        print('%s \t\t%s' % (net2.name, net2.get_shape()))

    net = conv2d('conv3', net2, [3, 3, 256, 384], [384], [1, 1, 1, 1],
                 padding='SAME', reuse=reuse, trainable=trainable, wd=0.0005,
                 activation=None)
    if debug:
        print('%s \t\t%s' % (net.name, net.get_shape()))
    net = tf.layers.batch_normalization(net, axis=3, training=is_training,
                                        name='conv3_bn')
    if debug:
        print('%s%s' % (net.name, net.get_shape()))
    net = tf.nn.relu(net, name='relu3')
    if debug:
        print('%s \t\t%s' % (net.name, net.get_shape()))

    net = conv2d('conv4', net, [3, 3, 384, 384], [384], [1, 1, 1, 1],
                 padding='SAME', reuse=reuse, trainable=trainable, wd=0.0005,
                 activation=None)
    if debug:
        print('%s \t\t%s' % (net.name, net.get_shape()))
    net = tf.layers.batch_normalization(net, axis=3, training=is_training,
                                        name='conv4_bn')
    if debug:
        print('%s%s' % (net.name, net.get_shape()))
    net4 = tf.nn.relu(net, name='relu4')
    if debug:
        print('%s \t\t%s' % (net4.name, net4.get_shape()))

    net = conv2d('conv5', net4, [3, 3, 384, 256], [256], [1, 1, 1, 1],
                 padding='SAME', reuse=reuse, trainable=trainable, wd=0.0005,
                 activation=None)
    if debug:
        print('%s \t\t%s' % (net.name, net.get_shape()))
    net = tf.layers.batch_normalization(net, axis=3, training=is_training,
                                        name='conv5_bn')
    if debug:
        print('%s%s' % (net.name, net.get_shape()))
    net = tf.nn.max_pool(net, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1],
                         padding='VALID', name='pool5')
    if debug:
        print('%s \t\t%s' % (net.name, net.get_shape()))
    net = tf.nn.relu(net, name='relu5')
    if debug:
        print('%s \t\t%s' % (net.name, net.get_shape()))

    net = tf.layers.batch_normalization(net, axis=1, training=is_training,
                                        name='fc6_bn')
    if debug:
        print('%s%s' % (net.name, net.get_shape()))
    net = fc('fc6', net, [6*8*256, 4096], [4096], reuse=reuse,
             activation='relu', trainable=trainable)
    if debug:
        print('%s \t\t%s' % (net.name, net.get_shape()))
    net = tf.nn.dropout(net, keep_drop)
    if debug:
        print('%s \t\t%s' % (net.name, net.get_shape()))

    net = tf.layers.batch_normalization(net, axis=1, training=is_training,
                                        name='fc7_bn')
    if debug:
        print('%s%s' % (net.name, net.get_shape()))
    net = fc('fc7', net, [4096, 6*8*256], [6*8*256], reuse=reuse,
             activation='relu', trainable=trainable)
    if debug:
            print('%s \t\t%s' % (net.name, net.get_shape()))
    net = tf.nn.dropout(net, keep_drop)
    if debug:
        print('%s \t%s' % (net.name, net.get_shape()))

    net = tf.reshape(net, [-1, 6, 8, 256])
    if debug:
        print('%s \t\t%s' % (net.name, net.get_shape()))

    net = tf.image.resize_images(net, size=[13, 17],
                                 method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    if debug:
        print('%s %s' % (net.name, net.get_shape()))
    net = conv2d('up1', net, [3, 3, 256, 384], [384], [1, 1, 1, 1],
                 padding='SAME', reuse=reuse, trainable=trainable, wd=0.0005,
                 activation=None)
    # net = conv2d('up1_1', net, [3, 3, 384, 384], [384], [1, 1, 1, 1],
    #              padding='SAME', reuse=reuse, trainable=trainable, wd=0.0005,
    #              activation=None)
    if debug:
        print('%s \t\t%s' % (net.name, net.get_shape()))
    net = tf.layers.batch_normalization(net, axis=3, training=is_training,
                                        name='up1_bn')
    if debug:
        print('%s%s' % (net.name, net.get_shape()))
    net = tf.nn.relu(net, name='up1_relu')
    if debug:
        print('%s \t\t%s' % (net.name, net.get_shape()))

    # residual
    net = net + net4
    if debug:
        print('%s \t\t\t%s' % (net.name, net.get_shape()))
    # net = tf.image.resize_images(net, size=[23, 32],
    #                              method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    # if debug:
    #     print('%s \t%s' % (net.name, net.get_shape()))
    net = conv2d('up2', net, [3, 3, 384, 256], [256], [1, 1, 1, 1],
                 padding='SAME', reuse=reuse, trainable=trainable, wd=0.0005,
                 activation=None)
    # net = conv2d('up2_1', net, [3, 3, 256, 256], [256], [1, 1, 1, 1],
    #              padding='SAME', reuse=reuse, trainable=trainable, wd=0.0005,
    #              activation=None)
    if debug:
        print('%s \t\t%s' % (net.name, net.get_shape()))
    net = tf.layers.batch_normalization(net, axis=3, training=is_training,
                                        name='up2_bn')
    if debug:
        print('%s%s' % (net.name, net.get_shape()))
    net = tf.nn.relu(net, name='up2_relu')
    if debug:
        print('%s \t\t%s' % (net.name, net.get_shape()))

    # residual
    net = net + net2

    if debug:
        print('%s \t\t%s' % (net.name, net.get_shape()))
    net = tf.image.resize_images(net, size=[27, 36],
                                 method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    if debug:
        print('%s %s' % (net.name, net.get_shape()))
    net = conv2d('up3', net, [3, 3, 256, 96], [96], [1, 1, 1, 1],
                 padding='SAME', reuse=reuse, trainable=trainable, wd=0.0005,
                 activation=None)
    # net = conv2d('up3_1', net, [3, 3, 96, 96], [96], [1, 1, 1, 1],
    #              padding='SAME', reuse=reuse, trainable=trainable, wd=0.0005,
    #              activation=None)
    if debug:
        print('%s \t\t%s' % (net.name, net.get_shape()))
    net = tf.layers.batch_normalization(net, axis=3, training=is_training,
                                        name='up3_bn')
    if debug:
        print('%s%s' % (net.name, net.get_shape()))
    net = tf.nn.relu(net, name='up3_relu')
    if debug:
        print('%s \t\t%s' % (net.name, net.get_shape()))

    # residual
    net = net + net1

    if debug:
        print('%s \t\t%s' % (net.name, net.get_shape()))

    net = tf.image.resize_images(net, size=[55, 74],
                                 method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    if debug:
        print('%s %s' % (net.name, net.get_shape()))
    net = conv2d('up4', net, [3, 3, 96, 48], [48], [1, 1, 1, 1],
                 padding='SAME', reuse=reuse, trainable=trainable, wd=0.0005,
                 activation=None)
    # net = conv2d('up4_1', net, [3, 3, 48, 48], [48], [1, 1, 1, 1],
    #              padding='SAME', reuse=reuse, trainable=trainable, wd=0.0005,
    #              activation=None)
    if debug:
        print('%s \t\t%s' % (net.name, net.get_shape()))
    net = tf.layers.batch_normalization(net, axis=3, training=is_training,
                                        name='up4_bn')
    if debug:
        print('%s%s' % (net.name, net.get_shape()))
    net = tf.nn.relu(net, name='up4_relu')
    if debug:
        print('%s \t\t%s' % (net.name, net.get_shape()))

    net = tf.image.resize_images(net, size=[111, 150],
                                 method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    if debug:
        print('%s %s' % (net.name, net.get_shape()))
    net = conv2d('up5', net, [3, 3, 48, 24], [24], [1, 1, 1, 1],
                 padding='SAME', reuse=reuse, trainable=trainable, wd=0.0005,
                 activation=None)
    # net = conv2d('up5_1', net, [3, 3, 24, 24], [24], [1, 1, 1, 1],
    #              padding='SAME', reuse=reuse, trainable=trainable, wd=0.0005,
    #              activation=None)
    if debug:
        print('%s \t\t%s' % (net.name, net.get_shape()))
    net = tf.layers.batch_normalization(net, axis=3, training=is_training,
                                        name='up5_bn')
    if debug:
        print('%s%s' % (net.name, net.get_shape()))
    net = tf.nn.relu(net, name='up5_relu')
    if debug:
        print('%s \t\t%s' % (net.name, net.get_shape()))
    net = tf.image.resize_images(net, size=[228, 304],
                                 method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    if debug:
        print('%s %s' % (net.name, net.get_shape()))
    net = conv2d('up6', net, [3, 3, 24, 1], [1], [1, 1, 1, 1],
                 padding='SAME', reuse=reuse, trainable=trainable, wd=0.0005,
                 activation=None)
    # net = conv2d('up6_1', net, [3, 3, 1, 1], [1], [1, 1, 1, 1],
    #              padding='SAME', reuse=reuse, trainable=trainable, wd=0.0005,
    #              activation=None)

    if debug:
        print('%s \t\t%s' % (net.name, net.get_shape()))
    return net

In [4]:
with tf.Graph().as_default():
    ds = dataset.DataSet(FLAGS.batch_size)
    images, depths, _ = ds.csv_inputs(FLAGS.train_file,
                                               target_size=[228, 304])
    keep_drop = tf.placeholder(tf.float32)
    encoder_model = model(images, is_training=True, keep_drop=keep_drop,
                          debug=True)

conv1/conv1:0 		(8, 55, 74, 96)
conv1_bn/batchnorm/add_1:0(8, 55, 74, 96)
pool1:0 		(8, 27, 36, 96)
relu1:0 		(8, 27, 36, 96)
conv2/conv2:0 		(8, 27, 36, 256)
conv2_bn/batchnorm/add_1:0(8, 27, 36, 256)
pool2:0 		(8, 13, 17, 256)
relu2:0 		(8, 13, 17, 256)
conv3/conv3:0 		(8, 13, 17, 384)
conv3_bn/batchnorm/add_1:0(8, 13, 17, 384)
relu3:0 		(8, 13, 17, 384)
conv4/conv4:0 		(8, 13, 17, 384)
conv4_bn/batchnorm/add_1:0(8, 13, 17, 384)
relu4:0 		(8, 13, 17, 384)
conv5/conv5:0 		(8, 13, 17, 256)
conv5_bn/batchnorm/add_1:0(8, 13, 17, 256)
pool5:0 		(8, 6, 8, 256)
relu5:0 		(8, 6, 8, 256)
fc6_bn/batchnorm/add_1:0(8, 6, 8, 256)
fc6/fc6_1:0 		(8, 4096)
dropout/mul:0 		(8, 4096)
fc7_bn/batchnorm/add_1:0(8, 4096)
fc7/fc7_1:0 		(8, 12288)
dropout_1/mul:0 	(8, 12288)
Reshape_1:0 		(8, 6, 8, 256)
ResizeNearestNeighbor:0 (8, 13, 17, 256)
up1/up1:0 		(8, 13, 17, 384)
up1_bn/batchnorm/add_1:0(8, 13, 17, 384)
up1_relu:0 		(8, 13, 17, 384)
add:0 			(8, 13, 17, 384)
up2/up2:0 		(8, 13, 17, 256)
up2_bn/batc

In [13]:
def dense_model(inputs, is_training, reuse=False, keep_drop=0.5, trainable=True,
          debug=False):
    """Create an upscaled Eigen coarse model.

    inputs.get_shape() = TensorShape([Dimension(8), Dimension(228),
                                      Dimension(304), Dimension(3)])
    """
    # Normalize.
    inputs = (inputs - dataset.IMAGES_MEAN) * dataset.IMAGES_ISTD
    net = conv2d('conv1', inputs, [11, 11, 3, 96], [96], [1, 4, 4, 1],
                 padding='VALID', reuse=reuse, trainable=trainable, wd=0.0005,
                 activation=None)
    if debug:
        print('%s \t\t%s' % (net.name, net.get_shape()))
    net = tf.layers.batch_normalization(net, axis=3, training=is_training,
                                        name='conv1_bn')
    if debug:
        print('%s%s' % (net.name, net.get_shape()))
    net = tf.nn.max_pool(net, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1],
                         padding='VALID', name='pool1')
    if debug:
        print('%s \t\t%s' % (net.name, net.get_shape()))
    net1 = tf.nn.relu(net, name='relu1')
    if debug:
        print('%s \t\t%s' % (net1.name, net1.get_shape()))

    net = conv2d('conv2', net1, [5, 5, 96, 256], [256], [1, 1, 1, 1],
                 padding='SAME', reuse=reuse, trainable=trainable, wd=0.0005,
                 activation=None)
    if debug:
        print('\n%s \t\t%s' % (net.name, net.get_shape()))
    net = tf.layers.batch_normalization(net, axis=3, training=is_training,
                                        name='conv2_bn')
    if debug:
        print('%s%s' % (net.name, net.get_shape()))
    net = tf.nn.max_pool(net, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1],
                         padding='VALID', name='pool2')
    if debug:
        print('%s \t\t%s' % (net.name, net.get_shape()))
    net2 = tf.nn.relu(net, name='relu2')
    if debug:
        print('%s \t\t%s' % (net2.name, net2.get_shape()))

    net = conv2d('conv3', net2, [3, 3, 256, 384], [384], [1, 1, 1, 1],
                 padding='SAME', reuse=reuse, trainable=trainable, wd=0.0005,
                 activation=None)
    if debug:
        print('\n%s \t\t%s' % (net.name, net.get_shape()))
    net = tf.layers.batch_normalization(net, axis=3, training=is_training,
                                        name='conv3_bn')
    if debug:
        print('%s%s' % (net.name, net.get_shape()))
    net = tf.nn.relu(net, name='relu3')
    if debug:
        print('%s \t\t%s' % (net.name, net.get_shape()))

    net = conv2d('conv4', net, [3, 3, 384, 384], [384], [1, 1, 1, 1],
                 padding='SAME', reuse=reuse, trainable=trainable, wd=0.0005,
                 activation=None)
    if debug:
        print('\n%s \t\t%s' % (net.name, net.get_shape()))
    net = tf.layers.batch_normalization(net, axis=3, training=is_training,
                                        name='conv4_bn')
    if debug:
        print('%s%s' % (net.name, net.get_shape()))
    net4 = tf.nn.relu(net, name='relu4')
    if debug:
        print('%s \t\t%s' % (net4.name, net4.get_shape()))

    net = conv2d('conv5', net4, [3, 3, 384, 256], [256], [1, 1, 1, 1],
                 padding='SAME', reuse=reuse, trainable=trainable, wd=0.0005,
                 activation=None)
    if debug:
        print('\n%s \t\t%s' % (net.name, net.get_shape()))
    net = tf.layers.batch_normalization(net, axis=3, training=is_training,
                                        name='conv5_bn')
    if debug:
        print('%s%s' % (net.name, net.get_shape()))
    net = tf.nn.max_pool(net, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1],
                         padding='VALID', name='pool5')
    if debug:
        print('%s \t\t%s' % (net.name, net.get_shape()))
    net = tf.nn.relu(net, name='relu5')
    if debug:
        print('%s \t\t%s' % (net.name, net.get_shape()))

    net = tf.layers.batch_normalization(net, axis=1, training=is_training,
                                        name='fc6_bn')
    if debug:
        print('\n%s%s' % (net.name, net.get_shape()))
    net = fc('fc6', net, [6*8*256, 4096], [4096], reuse=reuse,
             activation='relu', trainable=trainable)
    if debug:
        print('%s \t\t%s' % (net.name, net.get_shape()))
    net = tf.nn.dropout(net, keep_drop)
    if debug:
        print('%s \t\t%s' % (net.name, net.get_shape()))

    net = tf.layers.batch_normalization(net, axis=1, training=is_training,
                                        name='fc7_bn')
    if debug:
        print('\n%s%s' % (net.name, net.get_shape()))
    net = fc('fc7', net, [4096, 6*8*256], [6*8*256], reuse=reuse,
             activation='relu', trainable=trainable)
    if debug:
            print('%s \t\t%s' % (net.name, net.get_shape()))
    net = tf.nn.dropout(net, keep_drop)
    if debug:
        print('%s \t%s' % (net.name, net.get_shape()))

    net = tf.reshape(net, [-1, 6, 8, 256])
    if debug:
        print('%s \t\t%s' % (net.name, net.get_shape()))

    net = tf.image.resize_images(net, size=[13, 17],
                                 method=tf.image.ResizeMethod.BILINEAR)
    if debug:
        print('\n%s %s' % (net.name, net.get_shape()))

    # upscale conv layer 1
    net = conv2d('up1', net, [3, 3, 256, 384], [384], [1, 1, 1, 1],
                 padding='SAME', reuse=reuse, trainable=trainable, wd=0.0005,
                 activation=None)
    if debug:
        print('%s \t\t%s' % (net.name, net.get_shape()))
    net = tf.layers.batch_normalization(net, axis=3, training=is_training,
                                        name='up1_bn')
    if debug:
        print('%s%s' % (net.name, net.get_shape()))
    unet1 = tf.nn.relu(net, name='up1_relu')
    if debug:
        print('%s \t\t%s' % (unet1.name, unet1.get_shape()))

    # residual
    net = net4 + unet1
    if debug:
        print('%s \t\t\t%s' % (net.name, net.get_shape()))

    # upscale conv layer 2
    net = conv2d('up2', net, [3, 3, 384, 256], [256], [1, 1, 1, 1],
                 padding='SAME', reuse=reuse, trainable=trainable, wd=0.0005,
                 activation=None)
    if debug:
        print('\n%s \t\t%s' % (net.name, net.get_shape()))
    net = tf.layers.batch_normalization(net, axis=3, training=is_training,
                                        name='up2_bn')
    if debug:
        print('%s%s' % (net.name, net.get_shape()))
    unet2 = tf.nn.relu(net, name='up2_relu')
    if debug:
        print('%s \t\t%s' % (unet2.name, unet2.get_shape()))

    # skips
    unet1_ = conv2d('up3_unet1_skip', unet1, [3, 3, 384, 256], [256],
                    [1, 1, 1, 1], padding='SAME', reuse=reuse,
                    trainable=trainable, wd=0.0005, activation=None)
    if debug:
        print('%s%s' % (unet1_.name, unet1_.get_shape()))

    # residual
    net = net2 + unet1_ + unet2

    if debug:
        print('%s \t\t%s' % (net.name, net.get_shape()))
    net = tf.image.resize_images(net, size=[27, 36],
                                 method=tf.image.ResizeMethod.BILINEAR)
    if debug:
        print('%s %s' % (net.name, net.get_shape()))

    # upscale conv layer 3
    net = conv2d('up3', net, [3, 3, 256, 96], [96], [1, 1, 1, 1],
                 padding='SAME', reuse=reuse, trainable=trainable, wd=0.0005,
                 activation=None)
    if debug:
        print('\n%s \t\t%s' % (net.name, net.get_shape()))
    net = tf.layers.batch_normalization(net, axis=3, training=is_training,
                                        name='up3_bn')
    if debug:
        print('%s%s' % (net.name, net.get_shape()))
    unet3 = tf.nn.relu(net, name='up3_relu')
    if debug:
        print('%s \t\t%s' % (unet3.name, unet3.get_shape()))

    # skips
    unet1_ = tf.image.resize_images(unet1, size=[27, 36],
                                    method=tf.image.ResizeMethod.BILINEAR)
    unet1_ = conv2d('up4_unet1_skip', unet1_, [3, 3, 384, 96], [96],
                    [1, 1, 1, 1], padding='SAME', reuse=reuse,
                    trainable=trainable, wd=0.0005, activation=None)
    if debug:
        print('%s%s' % (unet1_.name, unet1_.get_shape()))
    unet2_ = tf.image.resize_images(unet2, size=[27, 36],
                                    method=tf.image.ResizeMethod.BILINEAR)
    unet2_ = conv2d('up4_unet2_skip', unet2_, [3, 3, 256, 96], [96],
                    [1, 1, 1, 1], padding='SAME', reuse=reuse,
                    trainable=trainable, wd=0.0005, activation=None)
    if debug:
        print('%s%s' % (unet2_.name, unet2_.get_shape()))

    # residual
    net = net1 + unet1_ + unet2_ + unet3

    if debug:
        print('%s \t\t%s' % (net.name, net.get_shape()))

    net = tf.image.resize_images(net, size=[55, 74],
                                 method=tf.image.ResizeMethod.BILINEAR)
    if debug:
        print('%s %s' % (net.name, net.get_shape()))

    # upscale conv layer 4
    net = conv2d('up4', net, [3, 3, 96, 48], [48], [1, 1, 1, 1],
                 padding='SAME', reuse=reuse, trainable=trainable, wd=0.0005,
                 activation=None)
    if debug:
        print('\n%s \t\t%s' % (net.name, net.get_shape()))
    net = tf.layers.batch_normalization(net, axis=3, training=is_training,
                                        name='up4_bn')
    if debug:
        print('%s%s' % (net.name, net.get_shape()))
    unet4 = tf.nn.relu(net, name='up4_relu')
    if debug:
        print('%s \t\t%s' % (unet4.name, unet4.get_shape()))

    # skips
    unet1_ = tf.image.resize_images(unet1, size=[55, 74],
                                    method=tf.image.ResizeMethod.BILINEAR)
    unet1_ = conv2d('up5_unet1_skip', unet1_, [3, 3, 384, 48], [48],
                    [1, 1, 1, 1], padding='SAME', reuse=reuse,
                    trainable=trainable, wd=0.0005, activation=None)
    if debug:
        print('%s%s' % (unet1_.name, unet1_.get_shape()))
    unet2_ = tf.image.resize_images(unet2, size=[55, 74],
                                    method=tf.image.ResizeMethod.BILINEAR)
    unet2_ = conv2d('up5_unet2_skip', unet2_, [3, 3, 256, 48], [48],
                    [1, 1, 1, 1], padding='SAME', reuse=reuse,
                    trainable=trainable, wd=0.0005, activation=None)
    if debug:
        print('%s%s' % (unet2_.name, unet2_.get_shape()))
    unet3_ = tf.image.resize_images(unet3, size=[55, 74],
                                    method=tf.image.ResizeMethod.BILINEAR)
    unet3_ = conv2d('up5_unet3_skip', unet3_, [3, 3, 96, 48], [48],
                    [1, 1, 1, 1], padding='SAME', reuse=reuse,
                    trainable=trainable, wd=0.0005, activation=None)
    if debug:
        print('%s%s' % (unet3_.name, unet3_.get_shape()))

    # residual
    net = unet1_ + unet2_ + unet3_ + unet4
    if debug:
        print('%s \t\t%s' % (net.name, net.get_shape()))

    net = tf.image.resize_images(net, size=[111, 150],
                                 method=tf.image.ResizeMethod.BILINEAR)
    if debug:
        print('%s %s' % (net.name, net.get_shape()))

    # upscale conv layer 5
    net = conv2d('up5', net, [3, 3, 48, 24], [24], [1, 1, 1, 1],
                 padding='SAME', reuse=reuse, trainable=trainable, wd=0.0005,
                 activation=None)
    if debug:
        print('\n%s \t\t%s' % (net.name, net.get_shape()))
    net = tf.layers.batch_normalization(net, axis=3, training=is_training,
                                        name='up5_bn')
    if debug:
        print('%s%s' % (net.name, net.get_shape()))
    unet5 = tf.nn.relu(net, name='up5_relu')
    if debug:
        print('%s \t\t%s' % (unet5.name, unet5.get_shape()))

    # skips
    unet1_ = tf.image.resize_images(unet1, size=[111, 150],
                                    method=tf.image.ResizeMethod.BILINEAR)
    unet1_ = conv2d('up6_unet1_skip', unet1_, [3, 3, 384, 24], [24],
                    [1, 1, 1, 1], padding='SAME', reuse=reuse,
                    trainable=trainable, wd=0.0005, activation=None)
    if debug:
        print('%s%s' % (unet1_.name, unet1_.get_shape()))
    unet2_ = tf.image.resize_images(unet2, size=[111, 150],
                                    method=tf.image.ResizeMethod.BILINEAR)
    unet2_ = conv2d('up6_unet2_skip', unet2_, [3, 3, 256, 24], [24],
                    [1, 1, 1, 1], padding='SAME', reuse=reuse,
                    trainable=trainable, wd=0.0005, activation=None)
    if debug:
        print('%s%s' % (unet2_.name, unet2_.get_shape()))
    unet3_ = tf.image.resize_images(unet3, size=[111, 150],
                                    method=tf.image.ResizeMethod.BILINEAR)
    unet3_ = conv2d('up6_unet3_skip', unet3_, [3, 3, 96, 24], [24],
                    [1, 1, 1, 1], padding='SAME', reuse=reuse,
                    trainable=trainable, wd=0.0005, activation=None)
    if debug:
        print('%s%s' % (unet3_.name, unet3_.get_shape()))
    unet4_ = tf.image.resize_images(unet4, size=[111, 150],
                                    method=tf.image.ResizeMethod.BILINEAR)
    unet4_ = conv2d('up6_unet4_skip', unet4_, [3, 3, 48, 24], [24],
                    [1, 1, 1, 1], padding='SAME', reuse=reuse,
                    trainable=trainable, wd=0.0005, activation=None)
    if debug:
        print('%s%s' % (unet4_.name, unet4_.get_shape()))

    # residual
    net = unet1_ + unet2_ + unet3_ + unet4_ + unet5
    if debug:
        print('%s \t\t%s' % (net.name, net.get_shape()))

    net = tf.image.resize_images(net, size=[228, 304],
                                 method=tf.image.ResizeMethod.BILINEAR)
    if debug:
        print('%s %s' % (net.name, net.get_shape()))

    # upscale conv layer 6
    net = conv2d('up6', net, [3, 3, 24, 1], [1], [1, 1, 1, 1],
                 padding='SAME', reuse=reuse, trainable=trainable, wd=0.0005,
                 activation=None)

    if debug:
        print('\n%s \t\t%s' % (net.name, net.get_shape()))
    return net

In [14]:
with tf.Graph().as_default():
    ds = dataset.DataSet(FLAGS.batch_size)
    images, depths, _ = ds.csv_inputs(FLAGS.train_file,
                                               target_size=[228, 304])
    keep_drop = tf.placeholder(tf.float32)
    encoder_model = dense_model(images, is_training=True, keep_drop=keep_drop,
                          debug=True)

conv1/conv1:0 		(8, 55, 74, 96)
conv1_bn/batchnorm/add_1:0(8, 55, 74, 96)
pool1:0 		(8, 27, 36, 96)
relu1:0 		(8, 27, 36, 96)

conv2/conv2:0 		(8, 27, 36, 256)
conv2_bn/batchnorm/add_1:0(8, 27, 36, 256)
pool2:0 		(8, 13, 17, 256)
relu2:0 		(8, 13, 17, 256)

conv3/conv3:0 		(8, 13, 17, 384)
conv3_bn/batchnorm/add_1:0(8, 13, 17, 384)
relu3:0 		(8, 13, 17, 384)

conv4/conv4:0 		(8, 13, 17, 384)
conv4_bn/batchnorm/add_1:0(8, 13, 17, 384)
relu4:0 		(8, 13, 17, 384)

conv5/conv5:0 		(8, 13, 17, 256)
conv5_bn/batchnorm/add_1:0(8, 13, 17, 256)
pool5:0 		(8, 6, 8, 256)
relu5:0 		(8, 6, 8, 256)

fc6_bn/batchnorm/add_1:0(8, 6, 8, 256)
fc6/fc6_1:0 		(8, 4096)
dropout/mul:0 		(8, 4096)

fc7_bn/batchnorm/add_1:0(8, 4096)
fc7/fc7_1:0 		(8, 12288)
dropout_1/mul:0 	(8, 12288)
Reshape_1:0 		(8, 6, 8, 256)

ResizeBilinear_2:0 (8, 13, 17, 256)
up1/up1:0 		(8, 13, 17, 384)
up1_bn/batchnorm/add_1:0(8, 13, 17, 384)
up1_relu:0 		(8, 13, 17, 384)
add:0 			(8, 13, 17, 384)

up2/up2:0 		(8, 13, 17, 256)
up2_bn/b